In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data

2024-10-28 12:42:54.824186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 12:42:54.837852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 12:42:54.843775: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 12:42:54.856990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 12:42:56.632019: W tensorflow/compiler/tf2

## Data

In [2]:
dataset = load_adni_data()
x = dataset.x
y = dataset.y

In [3]:
x_all = []
for item in x:
    x_all.append(item.flatten('F'))
x_all = np.array(x_all)

In [4]:
# split dataset into train and test sets 80/20 with random seed 42
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_all, y, test_size=0.2, random_state=42)

In [5]:
# load pt file 
import torch 
path = '/work/users/d/d/ddinh/aaco/results/dataset_adni_rollout_first_075_part2.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [6]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [7]:
merge_action.shape

(201, 48)

In [8]:
model = XGBClassifier()
model.load_model('/work/users/d/d/ddinh/aaco/models/adni_part2.model')

In [9]:
pred = model.predict_proba(np.concatenate([x_val[:merge_action.shape[0]] * merge_action, merge_action], axis=1))

In [10]:
pred[1]

array([0.55885047, 0.43976015, 0.00138938], dtype=float32)

In [11]:
num_ts = y_val.shape[1]

location = []
for i in range(merge_action.shape[0]):
    b = merge_action[i]
    temp = -1
    for j in range(4):
        part = b[num_ts*j:num_ts*(j+1)]
        max_index = np.where(part == 1)[0]
        if len(max_index) > 0:
            max_index = max_index[-1]
            if max_index > temp:
                temp = max_index
    location.append(temp)
location = np.array(location)

In [12]:
y_temp = []
for i in range(y_val.shape[0]):
    y_location = y_val[i, location[i]]
    mask_nan = np.isnan(y_location)
    y_location[mask_nan] = 0
    y_temp.append(y_location)
y_temp = np.array(y_temp)

In [13]:
y_temp.shape

(201, 3)

In [14]:
np.mean(np.argmax(y_temp, 1) == np.argmax(pred, 1))

0.3781094527363184

In [15]:
from sklearn.metrics import roc_curve, auc
import numpy as np

n_classes = 3

# Dictionary to store fpr, tpr, and roc_auc for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calculate ROC curve and AUC for each class
for i in range(n_classes):
    # y_true[:, i] gives the binary labels for class `i`
    # y_pred[:, i] gives the predicted probabilities for class `i`
    fpr[i], tpr[i], _ = roc_curve(y_temp[:, i], np.round(pred)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Display the results
for i in range(n_classes):
    print(f"Class {i}: AUC = {roc_auc[i]:.2f}")

# average: 
print(f"Average: AUC = {np.mean(list(roc_auc.values())):.2f}")

Class 0: AUC = 0.59
Class 1: AUC = 0.51
Class 2: AUC = 0.56
Average: AUC = 0.55


In [16]:
np.sum(pred[0])

1.0

In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_macro = roc_auc_score(y_temp, pred, average="macro", multi_class="ovr")
roc_auc_macro


0.5897137949769529

In [18]:
from sklearn.metrics import average_precision_score

pr_auc_macro = average_precision_score(y_temp, pred, average="macro")
pr_auc_macro

0.37685855223761705

In [19]:
y_temp[0]

array([0., 1., 0.])

In [20]:
np.mean(np.argmax(np.round(pred), 1) == np.argmax(y_temp,1))

0.3681592039800995

In [21]:
np.mean(np.argmax(y_temp,1) == np.argmax(np.round(pred),1))

0.3681592039800995

In [22]:
PET = 1
MRI = 0.5

merge_action_reshaped = np.copy(merge_action).reshape(-1, num_ts, 4)

In [23]:
merge_action_reshaped.shape

(201, 12, 4)

In [24]:
merge_action_reshaped[:, :, 0].shape

(201, 12)

In [25]:
np.mean(np.sum(merge_action_reshaped[:, :, 0], 1))

6.641791

In [26]:
total = 0
for i in range(4):
    if i <= 1:
        total += np.mean(np.sum(merge_action_reshaped[:, :, i], 1)) * PET
    else:
        total += np.mean(np.sum(merge_action_reshaped[:, :, i], 1)) * MRI
total

9.840795814990997

In [27]:
np.sum(merge_action_reshaped[:, :, 0], 1)

array([ 4., 13., 10.,  4.,  4., 10.,  4., 12.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  5., 13.,  4., 14.,  4.,  5.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  5.,  4.,  4.,  6.,  4.,
       12., 11.,  9.,  4.,  4.,  4., 13.,  4.,  8., 10., 12.,  5., 11.,
        4.,  4.,  4.,  8.,  4.,  5.,  8.,  4.,  9.,  4., 11., 15.,  4.,
       12.,  4.,  7.,  4.,  5.,  4.,  9.,  4.,  4.,  4.,  6.,  4.,  4.,
        4.,  5.,  9.,  5.,  4.,  4.,  4.,  4., 13.,  4.,  9.,  4.,  4.,
        4., 15.,  4.,  5., 15.,  5.,  6.,  4.,  4., 15.,  4.,  4.,  4.,
        5.,  4., 14., 15.,  4., 15., 12., 12., 12.,  4.,  5.,  4., 15.,
        4.,  4.,  7., 11.,  4., 14.,  4.,  4., 15., 11.,  4., 14.,  4.,
        5.,  4.,  4.,  4.,  5.,  8.,  4.,  4.,  4.,  5.,  4., 11.,  4.,
        4.,  4.,  4., 13., 14.,  4., 13.,  4.,  4.,  4.,  4.,  4., 13.,
       13., 13.,  4., 13.,  4.,  4., 10.,  4., 11., 13.,  5., 15., 10.,
        4.,  4., 15., 14.,  4., 11.,  4.,  4.,  5.,  4., 15.,  4

In [28]:
total = 0
count = 0
for i in range(4):
    if i <= 1:
        total += np.sum(merge_action_reshaped[:, :, i]) * PET
    else:
        total += np.sum(merge_action_reshaped[:, :, i]) * MRI
    count += np.sum(merge_action_reshaped[:, :, i])
total / count

0.7998382531338455

In [29]:
total = 0
count = 0
for j in range(merge_action_reshaped.shape[0]):
    total_temp = 0
    
    for i in range(4):
        if i <= 1:
            total_temp += np.sum(merge_action_reshaped[j, :, i]) * PET
        else:
            total_temp += np.sum(merge_action_reshaped[j, :, i]) * MRI
            
    total += total_temp 
total / merge_action_reshaped.shape[0] 

9.840796019900498

In [ ]:
9.840796019900498
